In [1]:
import pandas as pd
import numpy as np
import os
import time
import datetime
import re

import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import tools
init_notebook_mode(connected=True)

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#### Funtion Start ####
def auto_correlation(serie, lag):
    '''auto_correlation caculate the 
    autocorrelation of an given np_arry, 
    Input: pd_Series(sorted in datetime), lag.
    Output: auto_correlation'''
    
    arr = serie.values
    return np.corrcoef(arr[:-lag], arr[lag:])[0,1]


#### Function End ####

In [3]:
df_ABR = pd.read_csv('./df_ABR.csv', index_col=0)
df_ABR.shape

(3789, 13)

In [4]:
df_ABR['Treated'] = df_ABR['Treated'].astype(float)
df_ABR.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3789 entries, 1 to 33
Data columns (total 13 columns):
Drug                     3789 non-null object
Treated                  3789 non-null float64
Repulls                  3789 non-null float64
Mortalities              3789 non-null float64
Treatment Success(%)     3789 non-null float64
Case Fatality Rate(%)    3789 non-null float64
$/Hd/Rx                  3789 non-null float64
Treatment $              3789 non-null float64
Month                    3789 non-null object
Year                     3789 non-null int64
Risk                     3789 non-null object
Season                   3789 non-null object
Datetime                 3789 non-null object
dtypes: float64(7), int64(1), object(5)
memory usage: 414.4+ KB


In [5]:
float_colmuns = df_ABR.columns[df_ABR.dtypes == "float64"]
risk_level = list(df_ABR['Risk'].unique())
list_drug = list(df_ABR['Drug'].unique())

In [6]:
df_ABR.head()

,Drug,Treated,Repulls,Mortalities,Treatment Success(%),Case Fatality Rate(%),$/Hd/Rx,Treatment $,Month,Year,Risk,Season,Datetime
1,300PROLA,0.0,0.0,0.0,0.00,0.0,0.00,0.00,January,2018,Low,Other,2018-01-01
2,A180,0.0,0.0,0.0,0.00,0.0,0.00,0.00,January,2018,Low,Other,2018-01-01
3,A180As2ndRx,0.0,0.0,0.0,0.00,0.0,0.00,0.00,January,2018,Low,Other,2018-01-01
4,Adspec,0.0,0.0,0.0,0.00,0.0,0.00,0.00,January,2018,Low,Other,2018-01-01
5,Advocin,142.0,23.0,14.0,0.84,0.1,36.13,5130.82,January,2018,Low,Other,2018-01-01


In [7]:
df_ABR.describe()

,Treated,Repulls,Mortalities,Treatment Success(%),Case Fatality Rate(%),$/Hd/Rx,Treatment $,Year
count,3789.000000,3789.000000,3789.000000,3789.000000,3789.000000,3789.000000,3.789000e+03,3789.000000
mean,292.170494,31.460808,141.971496,0.212610,0.029538,4.832887,9.523792e+03,2017.720507
std,1501.857752,168.188368,5364.739355,0.384274,0.083868,15.838350,9.136413e+04,0.448809
min,0.000000,0.000000,-6.000000,0.000000,0.000000,0.000000,0.000000e+00,2017.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,2017.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,2018.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,2018.000000
max,32814.000000,3933.000000,270134.000000,1.000000,1.000000,428.250000,4.107891e+06,2018.000000


In [13]:
normalized_ABR = (df_ABR[float_colmuns]-df_ABR[float_colmuns].mean())/df_ABR[float_colmuns].std()


In [49]:
bins = pd.cut(df_ABR['Treated'], 20)
df_ABR['Treated'].value_counts(bins=5).index.astype(str).to_list()

['(-32.815, 6562.8]',
 '(6562.8, 13125.6]',
 '(13125.6, 19688.4]',
 '(26251.2, 32814.0]',
 '(19688.4, 26251.2]']

In [53]:

for col in float_colmuns:
    freq_table = df_ABR[col].value_counts(bins=50)
    data = [go.Bar(x=freq_table.index.astype(str).to_list(),
                   y=freq_table
                  )
            ]

    # setting layout
    layout = go.Layout(
        autosize=False,
        title=col,
        width=900,
        height=600
    )

    #combine data and layout settings into fig
    fig = go.Figure(data=data, layout=layout)

    #inline plot(iplot for jupyter notebook)
    iplot(fig, filename='drug_freq_table')

In [20]:
## boxplot for var

#getting the data ready for plot
data = [go.Box(x=df_ABR[df_ABR[col]!=0][col],
              name=col)
        for col in float_colmuns
        ]

# setting layout
layout = go.Layout(
    autosize=False,
    title='plot var float',
    width=900,
    height=600
)

#combine data and layout settings into fig
fig = go.Figure(data=data, layout=layout)

#inline plot(iplot for jupyter notebook)
iplot(fig, filename='drug_freq_table')

In [456]:
## freq table plot for drugs 
drug_freq_table = df_ABR['Drug'].value_counts()

#getting the data ready for plot
data = [go.Pie(labels=drug_freq_table.index,
                values=drug_freq_table
             )
        ]

# setting layout
layout = go.Layout(
    autosize=False,
    width=900,
    height=600
)

#combine data and layout settings into fig
fig = go.Figure(data=data, layout=layout)

#inline plot(iplot for jupyter notebook)
iplot(fig, filename='drug_freq_table')

In [54]:
## pie cahrt for risk
risk_data = df_ABR['Risk'].value_counts()

#getting the data ready for plot
data = [go.Pie(labels=risk_data.index,
               values=risk_data
              )        
        ]
# setting layout
layout = go.Layout(
    autosize=False,
    width=400,
    height=400
)

#combine data and layout settings into fig
fig = go.Figure(data=data, layout=layout)

#inline plot(iplot for jupyter notebook)
iplot(fig, filename='drug_freq_table')

In [325]:
seaon_mean = df_ABR.groupby(['Season']).mean()

,Treated,Repulls,Mortalities,Treatment Success(%),Case Fatality Rate(%),$/Hd/Rx,Treatment $,Year
Season,,,,,,,,
Fall,337.581425,43.615776,19.519084,0.234173,0.030254,5.210000,8587.107672,2017.694656
Other,247.790293,26.223443,13.137821,0.204657,0.029098,4.613333,7890.007674,2017.750000
Spring,366.936508,33.761905,603.046398,0.213126,0.030024,5.056447,14779.494603,2017.666667


In [350]:
data1 = [go.Pie(labels=seaon_data.index,
               values=seaon_data,
               name='Counts')
    
        ]

fig = go.Figure(data1)
iplot(fig)

In [353]:
## pie cahrt for seasons
seaon_data = df_ABR['Season'].value_counts()
seaon_mean = df_ABR.groupby(['Season']).mean()


#getting the data ready for plot
data1 = [go.Bar(x=seaon_mean.index,
               y=seaon_mean[col],
                name= col)
         for col in seaon_mean.columns
        ]

trace2 = go.Bar(x=seaon_data.index,
               y=seaon_data,
               name='Counts')
    

## create fig for subplot
fig = tools.make_subplots(rows=1, cols=2)

#append data into subplot figure
fig.append_trace(data1[0], 1, 1) #1 ,1  <==> first row first column  
for i in range(1,len(data1)):
    fig.append_trace(data1[i], 1, 1)
fig.append_trace(trace2, 1, 2) #1 ,2  <==> first row second column  

# setting layout
fig['layout'].update(height=600, width=1000, title='i <3 annotations and subplots')

#inline plot(iplot for jupyter notebook)
iplot(fig, filename='drug_freq_table')

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



In [192]:
float_colmuns = df_ABR.columns[df_ABR.dtypes == "float64"]

Index(['Repulls', 'Mortalities', 'Treatment Success(%)',
       'Case Fatality Rate(%)', '$/Hd/Rx', 'Treatment $'],
      dtype='object')

In [193]:
## line cahrt for datetime

#getting the data ready for plot
input_ = df_ABR[(df_ABR['Drug']=='Advocin')
       & (df_ABR['Risk']=='Low')].sort_values('Datetime')

float_colmuns = df_ABR.columns[df_ABR.dtypes == "float64"]

data = [go.Scatter(x=input_['Datetime'],
                y=input_[col],
                name=col)
        for col in float_colmuns]
# setting layout
layout = go.Layout(
    autosize=False,
    width=700,
    height=400
)

#combine data and layout settings into fig
fig = go.Figure(data=data, layout=layout)

#inline plot(iplot for jupyter notebook)
iplot(fig, filename='line')

In [211]:
## line cahrt for datetime

#getting the data ready for plot
Advocin_low = df_ABR[(df_ABR['Drug']=='Advocin')
       & (df_ABR['Risk']=='Low')].sort_values('Datetime')

float_colmuns = df_ABR.columns[df_ABR.dtypes == "float64"]
risk_level = list(df_ABR['Risk'].unique())

drug_datas = [go.Scatter(x=df_ABR[(df_ABR['Drug']=='Advocin')
                                    & (df_ABR['Risk']==risk)].sort_values('Datetime')['Datetime'],
                        y=df_ABR[(df_ABR['Drug']=='Advocin')
                                   & (df_ABR['Risk']==risk)].sort_values('Datetime')[col],
                        name=col+risk)
              for risk in risk_level  
              for col in float_colmuns]
# setting layout
layout = go.Layout(
    autosize=False,
    width=700,
    height=400
)

#combine data and layout settings into fig
fig = go.Figure(data=data, layout=layout)

#inline plot(iplot for jupyter notebook)
iplot(fig, filename='line')

In [356]:
df_drug_mean = df_ABR.groupby(['Drug'])[float_colmuns].mean()
df_drug_mean.head()

,Treated,Repulls,Mortalities,Treatment Success(%),Case Fatality Rate(%),$/Hd/Rx,Treatment $
Drug,,,,,,,
300PROLA,1.463768,0.434783,0.028986,0.010145,0.000290,0.066087,6.674783
A180,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
A180As2ndRx,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Adspec,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Advocin,338.565217,27.768116,200.608696,0.626667,0.122319,26.910145,13599.912899


In [432]:
## Corr & heatmap
data = [go.Heatmap(z=df_drug.corr())]
fig['layout']['yaxis']['autorange'] = "reversed"

# setting layout
layout = go.Layout(
    autosize=False,
    width=866,
    height=400
)

#combine data and layout settings into fig
fig = go.Figure(data=data, layout=layout)
fig['layout']['yaxis']['autorange'] = "reversed"  #改变 heatmap 方向（对角线）


#inline plot(iplot for jupyter notebook)
iplot(fig, filename='heatmap for Advocin_low')
df_drug.corr()


##insterestingly Case Fatality Rate is highly corr with % sucess

,Treated,Repulls,Mortalities,Treatment Success(%),Case Fatality Rate(%),$/Hd/Rx,Treatment $
Treated,1.000000,0.965244,0.527549,0.530836,0.084660,0.675203,0.937583
Repulls,0.965244,1.000000,0.536359,0.587221,0.154047,0.737213,0.905392
Mortalities,0.527549,0.536359,1.000000,0.462950,0.277445,0.580010,0.600504
Treatment Success(%),0.530836,0.587221,0.462950,1.000000,0.775473,0.816924,0.459734
Case Fatality Rate(%),0.084660,0.154047,0.277445,0.775473,1.000000,0.576800,0.098422
$/Hd/Rx,0.675203,0.737213,0.580010,0.816924,0.576800,1.000000,0.694709
Treatment $,0.937583,0.905392,0.600504,0.459734,0.098422,0.694709,1.000000


In [227]:
df_drug

Index(['300PROLA', 'A180', 'A180As2ndRx', 'Adspec', 'Advocin',
       'AdvocinAs2ndRx', 'AlbonBolus', 'Baytril', 'BaytrilAs2ndRx',
       'BaytrilAs3rdRx', 'Bio-Mycin', 'Bio-MyicnAs3rdRx', 'Borgal',
       'Ceftiflex', 'Draxxin', 'DraxxinAs2ndRx', 'DraxxinAs3rdRx', 'Excede',
       'ExcedeAs2ndRx', 'ExcedeAs3rdRx', 'Excenel', 'Hexasol', 'LA-200',
       'MassRxBio-Mycin', 'MassRxDraxxin', 'MassRxDraxxin/Excede',
       'MassRxExcede', 'MassRxLA-200', 'MassRxMicotil', 'MassRxNaxcel',
       'MassRxNoromycin300LA', 'MassRxNuflor', 'MassRxZactran',
       'MassRxZactran/Excede', 'MassRxZuprevo', 'Micotil', 'MicotilAs2ndRx',
       'MicotilAs3rdRx', 'Naxcel', 'NaxcelAs2ndRx', 'NaxcelAs3rdRx', 'Nuflor',
       'NuflorAs2ndRx', 'NuflorAs3rdRx', 'NuflorGold', 'NuflorGoldAs2ndRx',
       'Oxy100', 'Polyflex', 'Pro-Pen-G', 'ResflorGold', 'ResflorGoldAs2ndRx',
       'Tetradure', 'Tylan', 'Zactran', 'Zuprevo'],
      dtype='object', name='Drug')

In [238]:
df_drug


drug_datas = [go.Bar(x=df_drug.index,
                        y=df_drug[col],
                        name=col)
              for col in float_colmuns[:]]
# setting layout
layout = go.Layout(
    autosize=False,
    width=1010,
    height=800
)

#combine data and layout settings into fig
fig = go.Figure(data=drug_datas, layout=layout)

#inline plot(iplot for jupyter notebook)
iplot(fig, filename='line')

In [254]:
df_ABR.groupby(['Drug','Risk'])[float_colmuns].mean().sort_values(by='Treated',ascending=False)

Treated      Repulls   Mortalities  \
Drug                 Risk                                                
Draxxin              Moderate   9950.428571   932.785714    585.214286   
MassRxZactran        Low        9367.285714   550.285714     34.357143   
Draxxin              High       9057.846154  1141.538462  14992.000000   
                     Low        8093.428571   368.500000    404.071429   
MassRxMicotil        Moderate   7900.214286   930.357143    119.928571   
MassRxZuprevo        Moderate   3443.000000   317.285714     54.857143   
MassRxZactran        Moderate   2671.500000   121.785714     18.642857   
Bio-Mycin            Moderate   2415.500000   302.214286     48.571429   
MassRxDraxxin        Moderate   1783.214286   288.357143     43.714286   
Nuflor               Moderate   1765.928571   374.857143    178.500000   
ResflorGold          Moderate   1688.071429   437.428571    195.500000   
Nuflor               High       1614.142857   333.857143    166.285714   
                     Low        1365.285714   215.000000    117.071429   
Excede               High       1164.692308   146.230769  20862.076923   
MassRxZactran        High       1152.000000   211.615385     19.076923   
MassRxDraxxin        High       1136.230769   119.692308     15.615385   
Zuprevo              Moderate    941.500000     6.214286     98.571429   
MassRxDraxxin        Low         929.357143    15.571429     23.642857   
Advocin              Moderate    856.428571    48.500000    118.785714   
ResflorGoldAs2ndRx   Moderate    821.357143    46.714286    114.857143   
MassRxMicotil        Low         775.071429    64.642857      5.285714   
BaytrilAs2ndRx       Low         688.500000     0.214286     73.071429   
ExcedeAs2ndRx        Moderate    670.071429    75.642857    103.142857   
MassRxZuprevo        High        665.571429   249.714286     11.142857   
MassRxExcede         Moderate    651.714286    71.857143     13.428571   
Draxxin              Natural     645.000000   177.714286     29.285714   
ResflorGold          Low         617.928571   111.214286     70.642857   
NuflorAs2ndRx        Moderate    546.785714   110.142857    100.071429   
Advocin              High        543.307692    66.923077    895.307692   
Excede               Moderate    472.785714   128.714286     59.714286   
...                                     ...          ...           ...   
MassRxZactran/Excede Low           0.000000     0.000000      0.000000   
                     Moderate      0.000000     0.000000      0.000000   
                     Natural       0.000000     0.000000      0.000000   
                     UltraHigh     0.000000     0.000000      0.000000   
MassRxMicotil        UltraHigh     0.000000     0.000000      0.000000   
MassRxLA-200         UltraHigh     0.000000     0.000000      0.000000   
ExcedeAs3rdRx        Natural       0.000000     0.000000      0.000000   
300PROLA             Low           0.000000     0.000000      0.000000   
ExcedeAs3rdRx        UltraHigh     0.000000     0.000000      0.000000   
Hexasol              High          0.000000     0.000000      0.000000   
                     Low           0.000000     0.000000      0.000000   
                     Moderate      0.000000     0.000000      0.000000   
                     Natural       0.000000     0.000000      0.000000   
                     UltraHigh     0.000000     0.000000      0.000000   
LA-200               Natural       0.000000     0.000000      0.000000   
                     UltraHigh     0.000000     0.000000      0.000000   
MassRxBio-Mycin      High          0.000000     0.000000      0.000000   
                     Low           0.000000     0.000000      0.000000   
                     Natural       0.000000     0.000000      0.000000   
                     UltraHigh     0.000000     0.000000      0.000000   
MassRxDraxxin        Natural       0.000000     0.000000      0.000000   
                     UltraHigh     0.000

In [300]:
pca_data = df_ABR.groupby(['Drug','Risk','Season','Year'])[float_colmuns].mean()
normalized_pca_data = (pca_data-pca_data.mean())/pca_data.std()
normalized_pca_data

Treated   Repulls  Mortalities  \
Drug     Risk      Season Year                                    
300PROLA High      Fall   2018 -0.198200 -0.203214    -0.040424   
                   Other  2017 -0.198200 -0.203214    -0.040424   
                          2018 -0.198200 -0.203214    -0.040424   
                   Spring 2017 -0.198200 -0.203214    -0.040424   
                          2018 -0.198200 -0.203214    -0.040424   
         Low       Fall   2017 -0.198200 -0.203214    -0.040424   
                          2018 -0.198200 -0.203214    -0.040424   
                   Other  2017 -0.198200 -0.203214    -0.040424   
                          2018 -0.198200 -0.203214    -0.040424   
                   Spring 2017 -0.198200 -0.203214    -0.040424   
                          2018 -0.198200 -0.203214    -0.040424   
         Moderate  Fall   2017 -0.198200 -0.203214    -0.040424   
                          2018 -0.198200 -0.203214    -0.040424   
                   Other  2017 -0.198200 -0.203214    -0.040424   
                          2018 -0.198200 -0.203214    -0.040424   
                   Spring 2017 -0.198200 -0.203214    -0.040424   
                          2018 -0.198200 -0.203214    -0.040424   
         Natural   Fall   2017 -0.140312 -0.046407    -0.039939   
                          2018 -0.198200 -0.203214    -0.040424   
                   Other  2017 -0.198200 -0.203214    -0.040424   
                          2018 -0.198200 -0.203214    -0.040424   
                   Spring 2017 -0.198200 -0.203214    -0.040424   
                          2018 -0.198200 -0.203214    -0.040424   
         UltraHigh Fall   2017 -0.198200 -0.203214    -0.040424   
                          2018 -0.198200 -0.203214    -0.040424   
                   Other  2017 -0.198200 -0.203214    -0.040424   
                          2018 -0.198200 -0.203214    -0.040424   
                   Spring 2017 -0.198200 -0.203214    -0.040424   
                          2018 -0.198200 -0.203214    -0.040424   
A180     High      Fall   2018 -0.198200 -0.203214    -0.040424   
...                                  ...       ...          ...   
Zuprevo  High      Fall   2017 -0.156361 -0.077768    -0.040181   
                          2018 -0.095607  0.068585    -0.036306   
                   Other  2017 -0.082425 -0.059474    -0.036912   
                          2018 -0.168301 -0.141362    -0.039818   
                   Spring 2017 -0.011354  0.272433    -0.027830   
                          2018 -0.109362 -0.075155    -0.037517   
         Low       Fall   2017 -0.191323 -0.187533    -0.040424   
                          2018 -0.179860 -0.187533    -0.040303   
                   Other  2017 -0.198200 -0.203214    -0.040424   
                          2018 -0.191609 -0.184048    -0.040101   
                   Spring 2017 -0.198200 -0.203214    -0.040424   
                          2018 -0.174128 -0.132651    -0.038728   
         Moderate  Fall   2017 -0.030268 -0.182306    -0.036549   
                          2018  1.086511 -0.069928     0.016249   
                   Other  2017  0.101843 -0.169239    -0.025287   
                          2018  0.283913 -0.197116    -0.017537   
                   Spring 2017 -0.179860 -0.203214    -0.040424   
                          2018  0.454902 -0.171852    -0.015720   
         Natural   Fall   2017 -0.198200 -0.203214    -0.040424   
                          2018 -0.198200 -0.203214    -0.040424   
                   Other  2017 -0.198200 -0.203214    -0.040424   
                          2018 -0.198200 -0.203214    -0.040424   
                   Spring 2017 -0.198200 -0.203214    -0.040424   
                          2018 -0.198200 -0.203214    -0.040424   
         UltraHigh Fall   2017 -0.183872 -0.203214    -0.039697   
                          2018 -0.198200 -0.203214    -0.040424   
                   Other  2017 -0.198200 -0.203214    -0.040424   
             

In [433]:
from sklearn.decomposition import PCA

## pca 3 component

pca = PCA(n_components=3)
pca.fit(normalized_pca_data)
PCA(copy=True, iterated_power='auto', n_components=3, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)
print(pca.explained_variance_ratio_)  
print(pca.singular_values_)  

pca_3 = pca.transform(normalized_pca_data)


data = [go.Scatter3d(x=pca_3[:,0],
                     y=pca_3[:,1],
                     z=pca_3[:,2],
                    mode = 'markers' ##不要line
                    )
]

iplot(data,filename='suibian')


## no obvious group identify no need for futher clustering investigation of data

[0.46335938 0.21952737 0.15328366]
[72.12904505 49.64728953 41.48576728]


In [434]:
## pca two component
pca = PCA(n_components=2)
pca.fit(normalized_pca_data)
PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)
print(pca.explained_variance_ratio_)  
print(pca.singular_values_)  

pca_2 = pca.transform(normalized_pca_data)

data = [go.Scatter(x=pca_2[:,0],
                     y=pca_2[:,1],
                    mode = 'markers' ##不要line
                    )
]

iplot(data,filename='suibian')

[0.46335938 0.21952737]
[72.12904505 49.64728953]


In [395]:
df_ABR['Datetime'].nunique()

14

In [398]:
time_df_ABR = df_ABR.groupby(['Datetime']).mean().sort_values(by='Datetime')[float_colmuns]

In [409]:
## Auto_Correlation:[0]
for col in float_colmuns:
    print(col)
    for lag in range(1, 8):
        result = auto_correlation(time_df_ABR[col], lag)
        print(lag, '   ', result)

Treated
1     0.7696616300373672
2     0.4742777204969963
3     0.3851140744801261
4     0.15567278805318438
5     -0.18784168304487212
6     -0.5194096362362809
7     -0.3204481724204403
Repulls
1     0.7159830965044034
2     0.4319071497942416
3     0.15834258276252322
4     -0.05145911972815689
5     -0.4595207195747195
6     -0.8361752755513537
7     -0.7500574246379369
Mortalities
1     -0.08340951935688473
2     -0.08976948063031022
3     -0.09876647034023622
4     -0.113837418302413
5     -0.12725723386801038
6     -0.13966186781097426
7     0.7803971643688817
Treatment Success(%)
1     0.21180831775173986
2     0.05626318125801205
3     -0.23776304413864185
4     -0.36230091200345405
5     -0.38912968676471804
6     -0.5073356639614713
7     0.011931303861717394
Case Fatality Rate(%)
1     0.47728017604032624
2     -0.18768737807535218
3     -0.2654558139548846
4     -0.25291149440616445
5     -0.6367394821825819
6     -0.6573201014372784
7     0.022790837056268057
$/Hd/Rx
1   

In [495]:
## plot auto_colrelation:
max_lag = 9
data = [go.Scatter(x=[lag for lag in range(1,max_lag)],
                   y=[auto_correlation(time_df_ABR[col], lag) for lag in range(1,max_lag)],
                   name=col
                  )
       for col in float_colmuns]

iplot(data, filename='auto_correlation')

#plot suggested no enough data for a sound autocorrelation

In [493]:
## season
for col in float_colmuns:
    data =[go.Box(x=df_ABR[(df_ABR['Season']==season)
                          & (df_ABR['Treated']!=0)
                          & (df_ABR['Risk']==risk)][col],
                 name=season+'_'+risk)
          for risk in risk_level
          for season in ['Fall', 'Other','Spring']]

    
    ## create fig for subplot
    fig = tools.make_subplots(rows=2, cols=int(len(risk_level)/2))

    #append data into subplot figure
    i = 0
    for r in range(1,3):
        for c in range(1,3):
            for j in range(1,4):
                fig.append_trace(data[i], r, c)
                i = i +1


    # setting layout
    fig['layout'].update(height=500, width=1000, title=col)

    iplot(fig, filename='season')
    
### not much seasonal change

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]



This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]



This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]



This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]



This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]



This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]



This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]



In [538]:
len(float_colmuns)


7

In [562]:
risk_level

['Low', 'Natural', 'Moderate', 'High', 'UltraHigh']

In [566]:
class_code

{'Low': 0, 'Natural': 1, 'Moderate': 2, 'High': 3, 'UltraHigh': 4}

In [572]:
classes = risk_level
class_code = {classes[k]: k for k in range(len(risk_level))}
color_vals = [class_code[cl] for cl in df_ABR['Risk']]


In [573]:
trace1 = [go.Splom(dimensions=[dict(label=col,
                                 values=df_ABR[col])
                             for col in float_colmuns],
                 
                #default axes name assignment :
                #xaxes= ['x1','x2',  'x3'],
                #yaxes=  ['y1', 'y2', 'y3'], 
                marker=dict(color=color_vals,
                            size=7,
                            showscale=False,
                            line=dict(width=0.5,
                                      color='rgb(230,230,230)'))
                   
                )
         ]

In [574]:
axis = dict(showline=True,
          zeroline=False,)

layout = go.Layout(
    title='pair corr',
    dragmode='select',
    width=1000,
    height=700,
    autosize=False,
    hovermode='closest',
    xaxis1=dict(axis),
    xaxis2=dict(axis),
    xaxis3=dict(axis),
    xaxis4=dict(axis),
    xaxis5=dict(axis),
    xaxis6=dict(axis),
    xaxis7=dict(axis),
    yaxis1=dict(axis),
    yaxis2=dict(axis),
    yaxis3=dict(axis),
    yaxis4=dict(axis),
    yaxis5=dict(axis),
    yaxis6=dict(axis),
    yaxis7=dict(axis),
)

fig1 = dict(data=trace1, layout=layout)
iplot(fig1, filename='pair')
